# MNIST Single Layer Network with TensorBoard

注意：このノートブックはGPUの有無に関わらずPython 3.x および　TensorFlow 2.xにて1.x環境を実行可能にする設定（ないしTensorFlow 1.x） にて、オフラインにて稼動するよう変更しています。<br>
Google Colab環境にてオンラインで実行する場合、オリジナルサイトのコードを使って下さい。<br>
2020/02/04: By H.Nishiyama<br>
2021/11/08 追加・変更部分のみ日本語化<br>

このノートブックはTensorFlow 2.x (ないし1.x）を使います<br>
*もし、オンラインのCloabシステムを使うなら、オリジナルサイトのコードを使うか、次のセルを指示通りに修正して下さい。<br>
【オリジナルサイト】https://github.com/enakai00/colab_tfbook

In [1]:
#オンラインのCloabシステムを使う場合、下記の行頭の「###」を削除する。
###%tensorflow_version 2.x

####[MST-01]
Import modules and set random seeds.

In [2]:
import numpy as np
import shutil
  
import tensorflow as tf
from tensorflow.keras import layers, models, initializers, callbacks
from tensorflow.keras.datasets import mnist

np.random.seed(20190228)
#--- TensorFlow 1.x, を使う場合、下記のコメントアウト（###）を削除してください。
###tf.random.set_random_seed(20190228)
#--- For TensorFlow 2.x,用のコード。ensorFlow 1.x, を使う場合、下記をコメントアウトしてください。
tf.random.set_seed(20190228)

####[MST-02]
Download the MNIST dataset and store into NumPy arrays.

In [3]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape(
                  (len(train_images), 784)).astype('float32') / 255
test_images = test_images.reshape(
                  (len(test_images), 784)).astype('float32') / 255
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

####[MST-03]
Define a model with a single hidden layer.<br>
*Waring will be occured for some version of TensorFlow as follows;<br>
WARNING:tensorflow:From ... 

In [4]:
model = models.Sequential()
model.add(layers.Dense(1024, activation='relu', input_shape=(28*28,),
                       kernel_initializer=initializers.TruncatedNormal(),
                       name='hidden'))
model.add(layers.Dense(10, activation='softmax', name='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 1024)              803840    
_________________________________________________________________
softmax (Dense)              (None, 10)                10250     
Total params: 814,090
Trainable params: 814,090
Non-trainable params: 0
_________________________________________________________________


####[MST-04]
Compile the model using the Adam optimizer, and Cross entroy as a loss function.

In [5]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['acc'])

####[For local (off-line), windows system only!!]<br>
この部分は、Windowsでのローカル環境のみに適応しています。<br>
もし、上記ディレクトリ内に重要なファイルがあるようなら、予め別の場所に移動しておきましょう。<br>

In [6]:
import os
presentPath = os.getcwd()
driveLetter = presentPath[:presentPath.find(':')+1]
print(driveLetter)
!tree /A /F \tmp\log

c:
フォルダー パスの一覧
ボリューム シリアル番号は xxxx-xxxx です
C:\TMP\LOG
+---train
|   |   events.out.tfevents.1636380513.xxxxxxxx
|   |   events.out.tfevents.1636380514.xxxxxxxx
|   |   
|   \---plugins
|       \---profile
|           \---2021_11_08_14_08_34
|                   xxxxxxxx.input_pipeline.pb
|                   xxxxxxxx.kernel_stats.pb
|                   xxxxxxxx.memory_profile.json.gz
|                   xxxxxxxx.overview_page.pb
|                   xxxxxxxx.tensorflow_stats.pb
|                   xxxxxxxx.trace.json.gz
|                   xxxxxxxx.xplane.pb
|                   
\---validation
        events.out.tfevents.xxxxxxxx
        


####[MST-05]
Train the model with the callbacks option to store training logs.

In [7]:
# For online-colaboratory system, remove "###" from the top of following line.
###log_dir = '/tmp/log'
# For online-colaboratory system, add "###" to the top of following line. 
log_dir = "\\".join([driveLetter,'tmp\\log'])
shutil.rmtree(log_dir, ignore_errors=True)

tensorboard_callback = callbacks.TensorBoard(log_dir=log_dir,
                                             histogram_freq=1)
history = model.fit(train_images, train_labels,
                    validation_data=(test_images, test_labels),
                    batch_size=128, epochs=10,
                    callbacks=[tensorboard_callback])

Epoch 1/10
469/469 [==============================] - 3s 5ms/step - loss: 0.2360 - acc: 0.9314 - val_loss: 0.1190 - val_acc: 0.9655
Epoch 2/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0909 - acc: 0.9729 - val_loss: 0.0871 - val_acc: 0.9734
Epoch 3/10
469/469 [==============================] - 2s 3ms/step - loss: 0.0577 - acc: 0.9831 - val_loss: 0.0683 - val_acc: 0.9781
Epoch 4/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0388 - acc: 0.9884 - val_loss: 0.0684 - val_acc: 0.9789
Epoch 5/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0269 - acc: 0.9919 - val_loss: 0.0595 - val_acc: 0.9836
Epoch 6/10
469/469 [==============================] - 2s 5ms/step - loss: 0.0186 - acc: 0.9950 - val_loss: 0.0648 - val_acc: 0.9814
Epoch 7/10
469/469 [==============================] - 2s 4ms/step - loss: 0.0140 - acc: 0.9962 - val_loss: 0.0648 - val_acc: 0.9805
Epoch 8/10
469/469 [==============================] - 2s 4ms/step - loss: 0.

####[MST-06]
Install ngrok to run TensorBoard on Colaboratory.<br>
*For online-colaboratory system, remove "###" from the top of following line.

In [8]:
###!curl -OL https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
###!unzip -o ngrok-stable-linux-amd64.zip

####[MST-07]
Start TensorBoard and prepare the connection URL.<br>
*For online-colaboratory system, remove "###" from the top of following line.

In [9]:
###get_ipython().system_raw(
###    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
###    .format(log_dir)
###)
###get_ipython().system_raw('./ngrok http 6006 &')
###!curl -s http://localhost:4040/api/tunnels | python3 -c \
###    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Windows10であれば、Jupyterの"new->terminal"にて、ターミナル画面を開きます<br>
![Jupyter-open-new-Terminal.png](https://raw.githubusercontent.com/aujinen/colab_tfbook/master/docs/imgs/Jupyter-open-new-Terminal.png)


次に、下記の行を実行して得られるコマンドをターミナル内で実行しましょう。
ex. tensorboard --logdir "c:\tmp\log"

In [10]:
print('tensorboard --logdir "{}\\tmp\\log"'.format(driveLetter))

tensorboard --logdir "c:\tmp\log"


![open-TensorBoard-with-Shell.png](https://raw.githubusercontent.com/aujinen/colab_tfbook/master/docs/imgs/open-TensorBoard-with-Shell.png)

次に、下記URLを開きます。

http://localhost:6006/
<br><br>
もしくは、上記ターミナル内に表示されるURLを開いてください。
もし「No dashboards are active for the current data set.」と表示された場合には、「/tmp/log」が存在しているドライブを確認してください。<br>
終了するには、ターミナル内で【CTRL】+C を入力して停止後に、各画面を閉じてください。